In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
from matplotlib import rcParams, cycler
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
two_d_samples = th.randn(200,2)

plt.figure(figsize=(5,5))
plt.plot(var_to_np(two_d_samples)[:,0],
         var_to_np(two_d_samples)[:,1],
         ls='', marker='o');

one_d_samples = th.randn(200,1) * 1.5
samples = th.cat((one_d_samples, th.zeros_like(one_d_samples)), dim=1)
plt.plot(var_to_np(samples)[:,0],
         var_to_np(samples)[:,1],
         ls='', marker='o');

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c,),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c,),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2))
    )

In [ ]:
from reversible.util import set_random_seeds
set_random_seeds(20190227, True)
feature_model = nn.Sequential(
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
)

log_std = th.zeros(1, requires_grad=True)

optim_model = th.optim.Adam(list(feature_model.parameters()) + [log_std],
                          lr=1e-3)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    two_d_samples = th.randn(600,2) * 3

    one_d_samples = th.randn(600,1) * th.exp(log_std)
    samples = th.cat((one_d_samples, th.zeros_like(one_d_samples)), dim=1)
    in_samples = invert(feature_model, samples)
    sliced_loss = sliced_from_samples(two_d_samples, in_samples, 10, None)
    optim_model.zero_grad()
    sliced_loss.backward()
    optim_model.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Sliced Loss {:.2f}".format(sliced_loss.item()))
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(two_d_samples)[:,0],
                 var_to_np(two_d_samples)[:,1],
                 ls='', marker='o');

        plt.plot(var_to_np(in_samples)[:,0],
                 var_to_np(in_samples)[:,1],
                 ls='', marker='o');
        display(fig)
        plt.close(fig)

In [ ]:
ot_euclidean_loss_for_samples(two_d_samples, one_d_samples)

In [ ]:
ot_squared_l2_loss(two_d_samples, one_d_samples)

In [ ]:
th.exp(log_std)

In [ ]:
import ot
def ot_euclidean_loss_for_samples(samples_a, samples_b):
    diffs = samples_a.unsqueeze(1) - samples_b.unsqueeze(0)
    diffs = th.sqrt(th.clamp(th.sum(diffs * diffs, dim=2), min=1e-6))

    transport_mat = ot.emd([], [], var_to_np(diffs))
    # sometimes weird low values, try to prevent them
    transport_mat = transport_mat * (transport_mat > (1.0/(diffs.numel())))

    transport_mat = np_to_var(transport_mat, dtype=np.float32)
    #diffs, transport_mat = ensure_on_same_device(diffs, transport_mat)
    loss = th.sum(transport_mat * diffs)
    return loss


def ot_squared_l2_loss(samples_a, samples_b):
    diffs = samples_a.unsqueeze(1) - samples_b.unsqueeze(0)
    diffs = th.sum(diffs * diffs, dim=2)

    transport_mat = ot.emd([], [], var_to_np(diffs))
    # sometimes weird low values, try to prevent them
    transport_mat = transport_mat * (transport_mat > (1.0/(diffs.numel())))

    transport_mat = np_to_var(transport_mat, dtype=np.float32)
    #diffs, transport_mat = ensure_on_same_device(diffs, transport_mat)
    eps = 1e-8
    loss = th.sqrt(th.sum(transport_mat * diffs) + eps)
    return loss

In [ ]:
from reversible.util import set_random_seeds
set_random_seeds(20190227, True)
feature_model = nn.Sequential(
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
    rev_block(2,200),
)

log_std = th.zeros(1, requires_grad=True)

optim_model = th.optim.Adam(list(feature_model.parameters()) + [log_std],
                          lr=1e-3)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    two_d_samples = th.randn(200,2) * 3

    one_d_samples = th.randn(600,1) * th.exp(log_std)
    samples = th.cat((one_d_samples, th.zeros_like(one_d_samples)), dim=1)
    in_samples = invert(feature_model, samples)
    ot_loss = ot_squared_l2_loss(two_d_samples, one_d_samples)
    optim_model.zero_grad()
    ot_loss.backward()
    optim_model.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("OT Loss {:.2f}".format(ot_loss.item()))
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(two_d_samples)[:,0],
                 var_to_np(two_d_samples)[:,1],
                 ls='', marker='o');

        plt.plot(var_to_np(in_samples)[:,0],
                 var_to_np(in_samples)[:,1],
                 ls='', marker='o');
        display(fig)
        plt.close(fig)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    two_d_samples = th.randn(200,2) * 3

    one_d_samples = th.randn(600,1) * th.exp(log_std)
    samples = th.cat((one_d_samples, th.zeros_like(one_d_samples)), dim=1)
    in_samples = invert(feature_model, samples)
    ot_loss = ot_euclidean_loss_for_samples(two_d_samples, one_d_samples)
    optim_model.zero_grad()
    ot_loss.backward()
    optim_model.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("OT Loss {:.2f}".format(ot_loss.item()))
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(two_d_samples)[:,0],
                 var_to_np(two_d_samples)[:,1],
                 ls='', marker='o');

        plt.plot(var_to_np(in_samples)[:,0],
                 var_to_np(in_samples)[:,1],
                 ls='', marker='o');
        display(fig)
        plt.close(fig)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    two_d_samples = th.randn(600,2) * 3

    one_d_samples = th.randn(600,1) * th.exp(log_std)
    samples = th.cat((one_d_samples, th.zeros_like(one_d_samples)), dim=1)
    in_samples = invert(feature_model, samples)
    ot_loss = ot_euclidean_loss_for_samples(two_d_samples, one_d_samples)
    optim_model.zero_grad()
    ot_loss.backward()
    optim_model.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("OT Loss {:.2f}".format(ot_loss.item()))
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(two_d_samples)[:,0],
                 var_to_np(two_d_samples)[:,1],
                 ls='', marker='o');

        plt.plot(var_to_np(in_samples)[:,0],
                 var_to_np(in_samples)[:,1],
                 ls='', marker='o');
        display(fig)
        plt.close(fig)